## データセットの作成
データベースから各レースごとに最大24馬の過去五レースをとってくる
ない分は0埋め

race_id がキーの\[そのレースのdataframe, 過去レースのdataframe\]の辞書を作る

処理は置いといてとりあえず集める

In [37]:
import sqlite3
from contextlib import closing
import numpy as np
import pickle
import pandas as pd

dbpath = './data/horse.db'
with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
    c = conn.cursor()
    indexs = list(c.execute('pragma table_info(race)'))
    print(np.array(indexs))

[[0 'horse_id' 'int' 0 None 0]
 [1 'R' 'int' 0 None 0]
 [2 'head_count' 'int' 0 None 0]
 [3 'wakuban' 'int' 0 None 0]
 [4 'umaban' 'int' 0 None 0]
 [5 'popularity' 'int' 0 None 0]
 [6 'race_order' 'int' 0 None 0]
 [7 'burden_weight' 'int' 0 None 0]
 [8 'odds' 'float' 0 None 0]
 [9 'time_gap' 'float' 0 None 0]
 [10 'up' 'float' 0 None 0]
 [11 'weather' 'text' 0 None 0]
 [12 'baba' 'text' 0 None 0]
 [13 'raceid' 'int' 0 None 0]
 [14 'place' 'text' 0 None 0]
 [15 'pace0' 'float' 0 None 0]
 [16 'pace1' 'float' 0 None 0]
 [17 'race_type' 'text' 0 None 0]
 [18 'distance' 'int' 0 None 0]
 [19 'weight' 'int' 0 None 0]
 [20 'weight_change' 'int' 0 None 0]
 [21 'date' 'datetime' 0 None 0]
 [22 'time' 'datetime' 0 None 0]]


In [31]:
with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
    c = conn.cursor()
    [print(idx, set(c.execute(f'select {idx} from race'))) for idx in 'weather baba place race_type'.split(' ')]

weather {('晴',), ('小雪',), ('雪',), ('小雨',), ('曇',), ('雨',)}
baba {('稍',), ('良',), ('重',), ('不',)}
place {('名古屋',), ('福島',), ('小倉',), ('笠松',), ('中山',), ('函館',), ('高知',), ('園田',), ('門別',), ('大井',), ('京都',), ('東京',), ('浦和',), ('金沢',), ('船橋',), ('札幌',), ('川崎',), ('阪神',), ('盛岡',), ('佐賀',), ('新潟',), ('中京',), ('水沢',)}
race_type {('ダ',), ('障',), ('芝',)}


weather とbaba については連続値化, placeはis中山, race_typeはone-hot

In [34]:
for i in indexs:
    print(f'{i[1]}', end=',')

horse_id,R,head_count,wakuban,umaban,popularity,race_order,burden_weight,odds,time_gap,up,weather,baba,raceid,place,pace0,pace1,race_type,distance,weight,weight_change,date,time,

In [67]:
idxs = 'horse_id,R,head_count,wakuban,umaban,popularity,race_order,burden_weight,odds,time_gap,up,weather,baba,raceid,place,pace0,pace1,race_type,distance,weight,weight_change,date,time'.split(',')

In [94]:
with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
    c = conn.cursor()
    df = pd.DataFrame(c.execute('select * from race order by horse_id desc'), columns=idxs)
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'])
df_onehot = pd.get_dummies(df)
df_onehot

,horse_id,R,head_count,wakuban,umaban,popularity,race_order,burden_weight,odds,time_gap,...,place_福島,place_笠松,place_船橋,place_金沢,place_門別,place_阪神,place_高知,race_type_ダ,race_type_芝,race_type_障
0,2017110028,9,12,4,4,1,2.0,55.0,2.6,0.5,...,0,0,0,0,0,0,0,0,1,0
1,2017110028,5,14,3,3,3,1.0,54.0,4.1,-0.3,...,0,0,0,0,0,0,0,0,1,0
2,2017110028,4,18,2,4,2,3.0,54.0,4.4,0.2,...,0,0,0,0,0,0,0,0,1,0
3,2017110028,5,13,8,13,1,4.0,54.0,1.7,0.2,...,1,0,0,0,0,0,0,0,1,0
4,2017110026,9,12,5,6,1,3.0,55.0,2.4,0.2,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23524,2014106038,10,7,4,4,1,1.0,57.0,1.8,0.0,...,0,0,0,0,0,0,0,0,1,0
23525,2014106038,11,12,3,3,1,2.0,56.0,3.9,0.0,...,0,0,0,0,0,0,0,0,1,0
23526,2014106038,9,10,2,2,1,2.0,56.0,2.2,0.1,...,0,0,0,0,0,0,0,0,1,0
23527,2014106038,5,16,8,16,1,1.0,56.0,1.8,-0.2,...,0,0,0,0,0,0,0,0,1,0


In [124]:
df_onehot['date'][1:].reset_index()['date']

0       2019-10-26
1       2019-08-03
2       2019-07-14
3       2019-11-30
4       2019-08-31
           ...    
57537   2016-04-09
57538   2016-03-12
57539   2016-02-13
57540   2016-01-16
57541   2015-12-13
Name: date, Length: 57542, dtype: datetime64[ns]

In [169]:
df_onehot['interval'] = (df_onehot['date'] - df_onehot['date'][1:].reset_index()['date']).dt.days
df_onehot['time'] = df_onehot['time'].dt.second
df_onehot.loc[df_onehot['interval']<0, 'interval'] = 365
df_onehot

,horse_id,R,head_count,wakuban,umaban,popularity,race_order,burden_weight,odds,time_gap,...,place_笠松,place_船橋,place_金沢,place_門別,place_阪神,place_高知,race_type_ダ,race_type_芝,race_type_障,interval
0,2017110028,9,12,4,4,1,2.0,55.0,2.6,0.5,...,0,0,0,0,0,0,0,1,0,42.0
1,2017110028,5,14,3,3,3,1.0,54.0,4.1,-0.3,...,0,0,0,0,0,0,0,1,0,84.0
2,2017110028,4,18,2,4,2,3.0,54.0,4.4,0.2,...,0,0,0,0,0,0,0,1,0,20.0
3,2017110028,5,13,8,13,1,4.0,54.0,1.7,0.2,...,0,0,0,0,0,0,0,1,0,365.0
4,2017110026,9,12,5,6,1,3.0,55.0,2.4,0.2,...,0,0,0,0,0,0,0,1,0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57538,2013103907,5,16,6,11,6,3.0,56.0,30.7,0.3,...,0,0,0,0,0,0,0,1,0,28.0
57539,2013103907,5,16,7,13,9,6.0,56.0,27.8,0.7,...,0,0,0,0,0,0,0,1,0,28.0
57540,2013103907,4,16,4,8,8,4.0,56.0,29.7,0.9,...,0,0,0,0,0,0,0,1,0,28.0
57541,2013103907,5,16,3,5,10,7.0,56.0,136.8,0.7,...,0,0,0,0,0,0,0,1,0,34.0


In [146]:
df_onehot['interval'].max()

948.0

In [170]:
df_onehot['time']

0         9
1        22
2        56
3        12
4        59
         ..
57538     0
57539    17
57540    28
57541     2
57542    38
Name: time, Length: 57543, dtype: int64

そんなにないので適当に線形保管に決める

In [117]:
sorted(set(df_onehot[df_onehot['place_中山']==1]['raceid']))[::-1])

list

In [187]:
class RacedfLoopHandyman:
    def __init__(self, savearray, n_pre=5):
        self.checklist = []
        self.savearray = savearray
        self.n_pre = n_pre
    
    def data2data_pre(self, d):
        return d[[-1, 2, 3, 4, 7, 15, 16, 14, 1, 6, 8, 5, 18, 9, 12, 13, 10, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, -4, -3, -2]]
    
    def throw_data(self, d):
        new_checklist = []
        for race_idx, horse_idx, pre_idx, horse_id in self.checklist:
            if horse_id != d['horse_id']:
                continue
            if pre_idx == n_pre:
                continue
            self.savearray[race_idx, horse_idx, pre_idx] = self.data2data_pre(d)
            new_checklist.append((race_idx, horse_idx, pre_idx+1, horse_id))
        
    def new_nakayama(self, race_idx, horse_idx, horse_id):
        self.checklist.append((race_idx, horse_idx, 0, horse_id))

In [ ]:
# db=>df
with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
    c = conn.cursor()
    df = pd.DataFrame(c.execute('select * from race order by horse_id desc'), columns=idxs)
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'])
df_onehot = pd.get_dummies(df).interpolate()
# intervalを計算
df_onehot['interval'] = (df_onehot['date'] - df_onehot['date'][1:].reset_index()['date']).dt.days
df_onehot.loc[df_onehot['interval']<0, 'interval'] = 365
df_onehot['time'] = df_onehot['time'].dt.second
df_onehot['umaban'] -= 1
# 入力用ndarray の作成
n_race = len(df_onehot[df_onehot['place_中山']==1])
n_horse = 18
n_feature_the_day = 22
n_feature_pre = 30
n_pre = 5 
d_the_day = np.zeros((n_race, n_horse, n_feature_the_day))
d_pre = np.zeros((n_race, n_horse, n_pre, n_feature_pre))

raceid2idx = {raceid: idx for idx, raceid in enumerate(sorted(set(df_onehot[df_onehot['place_中山']==1]['raceid']))[::-1])}

def data2data_theday(d):
    return d[[-1, 2, 3, 4, 7, 15, 16, 14, 1, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, -4, -3, -2]]
    

handyman = RacedfLoopHandyman(d_pre, n_pre=n_pre)
for i in range(len(df_onehot)):
    d = df_onehot.iloc[i]
    handyman.throw_data(d)
    if d['place_中山'] == 1:
        raceidx = raceid2idx[d['raceid']]
        handyman.new_nakayama(raceidx, d['umaban'], d['horse_id'])
        d_the_day[raceidx, d['umaban']] = data2data_theday(d)

In [200]:
df_onehot[df_onehot['place_中山']==1][df_onehot[df_onehot['place_中山']==1]['head_count']>18]

,horse_id,R,head_count,wakuban,umaban,popularity,race_order,burden_weight,odds,time_gap,...,place_船橋,place_荒尾,place_金沢,place_門別,place_阪神,place_高知,race_type_ダ,race_type_芝,race_type_障,interval


In [189]:
df_onehot.iloc[i]

horse_id                  2017101585
R                                  5
head_count                        12
wakuban                            6
umaban                             8
popularity                        11
race_order                         7
burden_weight                     51
odds                            94.4
time_gap                         0.9
up                              34.2
raceid                  201905030105
pace0                           36.7
pace1                           34.3
distance                        1400
weight                           424
weight_change                      0
date             2019-06-01 00:00:00
time                              24
weather_小雨                         0
weather_晴                          1
weather_曇                          0
weather_雨                          0
baba_不                             0
baba_稍                             0
baba_良                             1
baba_重                             0
p

In [172]:
d =  df_onehot.iloc[1]
for hoge in enumerate(d.index):
    print(hoge)

(0, 'horse_id')
(1, 'R')
(2, 'head_count')
(3, 'wakuban')
(4, 'umaban')
(5, 'popularity')
(6, 'race_order')
(7, 'burden_weight')
(8, 'odds')
(9, 'time_gap')
(10, 'up')
(11, 'raceid')
(12, 'pace0')
(13, 'pace1')
(14, 'distance')
(15, 'weight')
(16, 'weight_change')
(17, 'date')
(18, 'time')
(19, 'weather_小雨')
(20, 'weather_小雪')
(21, 'weather_晴')
(22, 'weather_曇')
(23, 'weather_雨')
(24, 'weather_雪')
(25, 'baba_不')
(26, 'baba_稍')
(27, 'baba_良')
(28, 'baba_重')
(29, 'place_中京')
(30, 'place_中山')
(31, 'place_京都')
(32, 'place_佐賀')
(33, 'place_函館')
(34, 'place_名古屋')
(35, 'place_園田')
(36, 'place_大井')
(37, 'place_小倉')
(38, 'place_川崎')
(39, 'place_新潟')
(40, 'place_札幌')
(41, 'place_東京')
(42, 'place_水沢')
(43, 'place_浦和')
(44, 'place_盛岡')
(45, 'place_福島')
(46, 'place_笠松')
(47, 'place_船橋')
(48, 'place_金沢')
(49, 'place_門別')
(50, 'place_阪神')
(51, 'place_高知')
(52, 'race_type_ダ')
(53, 'race_type_芝')
(54, 'race_type_障')
(55, 'interval')


In [176]:
d[[55, 2, 3, 4, 7, 15, 16, 14, 1, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 52, 53, 54]]

22

In [177]:
d[[55, 2, 3, 4, 7, 15, 16, 14, 1, 6, 8, 5, 18, 9, 12, 13, 10, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 52, 53, 54]]

30

### A
入力: 

- 当日情報
    - 数値データ
        - 前のレースから何日
        - 頭数
        - 枠版
        - 馬番
        - 斤量
        - 重量
        - 重量変化
        - 距離
        - R 
    - カテゴリ(one-hot でok)
        - レース種類
        - 天気
        - 馬場
- 過去五レース(当日情報で扱うデータに加え)
    - 数値データ
        - 着順
        - odds
        - 人気
        - タイム
        - 着差
        - ペース
        - 上り
    - カテゴリ(one-hot でok)
    
出力: score(float)